# Visualization Example

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import sys
DIR = "/workspace/nflbigdatabowl2023"
sys.path.append(DIR)

In [9]:
import math
from ast import literal_eval

import pandas as pd
import matplotlib.pyplot as plt

from src.pipeline.flows.main import main_flow
from src.visualization.interactive_play_selector import create_interactive_play_selector

In [10]:
_ = main_flow(max_weeks=3, max_games=3, max_plays=3)

04:24:05.311 | INFO    | prefect.engine - Created flow run 'rustling-jackdaw' for flow 'main-flow'

04:24:05.479 | INFO    | Flow run 'rustling-jackdaw' - Created task run 'read_csv-3609c996-0' for task 'read_csv'

04:24:05.481 | INFO    | Flow run 'rustling-jackdaw' - Executing 'read_csv-3609c996-0' immediately...

04:24:05.773 | INFO    | Task run 'read_csv-3609c996-0' - Finished in state Completed()

04:24:05.814 | INFO    | Flow run 'rustling-jackdaw' - Created task run 'read_csv-3609c996-1' for task 'read_csv'

...

04:24:17.371 | INFO    | Task run 'write_csv-386fe2af-4' - Finished in state Completed()

04:24:17.397 | INFO    | Flow run 'rustling-jackdaw' - Created task run 'write_csv-386fe2af-5' for task 'write_csv'

04:24:17.399 | INFO    | Flow run 'rustling-jackdaw' - Executing 'write_csv-386fe2af-5' immediately...

04:24:17.466 | INFO    | Task run 'write_csv-386fe2af-5' - Finished in state Completed()

04:24:17.513 | INFO    | Flow run 'rustling-jackdaw' - Finished in state Completed('All states completed.')

In [5]:
df_plays_all = pd.read_csv(f"{DIR}/data/raw/plays.csv")
df_tracking_display_all = pd.read_csv(f"{DIR}/data/outputs/tracking_display.csv")
df_areas_all = pd.read_csv(f"{DIR}/data/outputs/pocket_areas.csv")
df_areas_all["pocket"] = df_areas_all["pocket"].apply(literal_eval)

In [6]:
create_interactive_play_selector(
    df_plays_all,
    df_tracking_display_all,
    df_areas_all,
    continuous_update=True
)

interactive(children=(Dropdown(description='Week', options=(1, 2, 3), value=1), Dropdown(description='Game ID'…